In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pickle
import os
import cv2
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.models import load_model
from tensorflow.keras import regularizers
from PIL import Image
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical

In [ ]:
train_images = []
train_labels = []
shape = (200,200)
train_path = '/content/drive/MyDrive/DeepLearning/Colab/ObjectDetection/Fruit_Dataset/train_zip/train'

for filename in os.listdir('/content/drive/MyDrive/DeepLearning/Colab/ObjectDetection/Fruit_Dataset/train_zip/train'):
    if filename.split('.')[1] == 'jpg':
        img = cv2.imread(os.path.join(train_path,filename))

        train_labels.append(filename.split('_')[0])

        img = cv2.resize(img,shape)

        train_images.append(img)

train_labels = pd.get_dummies(train_labels).values

train_images = np.array(train_images)

x_train,x_val,y_train,y_val = train_test_split(train_images,train_labels,random_state=1)

In [ ]:
test_images = []
test_labels = []
shape = (200,200)
test_path = '/content/drive/MyDrive/DeepLearning/Colab/ObjectDetection/Fruit_Dataset/test_zip/test'

for filename in os.listdir('/content/drive/MyDrive/DeepLearning/Colab/ObjectDetection/Fruit_Dataset/test_zip/test'):
    if filename.split('.')[1] == 'jpg':
        img = cv2.imread(os.path.join(test_path,filename))

        test_labels.append(filename.split('_')[0])

        img = cv2.resize(img,shape)

        test_images.append(img)

# Converting test_images to array
test_images = np.array(test_images)

In [ ]:
def cnn_model_v2():
    model = keras.Sequential([
        keras.Input(shape=(128, 128, 3)),

        layers.Conv2D(32, kernel_size=(3, 3), activation="relu", padding="same"),
        layers.BatchNormalization(),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),

        layers.Conv2D(64, kernel_size=(3, 3), activation="relu", padding="same"),
        layers.BatchNormalization(),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),

        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation="relu"),
        layers.Dropout(0.5),
        layers.Dense(128, activation="relu"),
        layers.Dropout(0.5),
        layers.Dense(4, activation="softmax")  # Adjust number of classes as needed
    ])

    return model

In [ ]:
model = cnn_model_v2()
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_20 (Conv2D)                   │ (None, 128, 128, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_10               │ (None, 128, 128, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_21 (Conv2D)                   │ (None, 128, 128, 32)        │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_10 (MaxPooling2D)      │ (None, 64, 64, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_22 (Conv2D)                   │ (None, 64, 64, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_11               │ (None, 64, 64, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_23 (Conv2D)                   │ (None, 64, 64, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_11 (MaxPooling2D)      │ (None, 32, 32, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_5           │ (None, 64)                  │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 256)                 │          16,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 4)                   │             516 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 116,004 (453.14 KB)

 Trainable params: 115,812 (452.39 KB)

 Non-trainable params: 192 (768.00 B)

In [ ]:
adam = Adam(learning_rate=0.001)
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(x_train,y_train,epochs=50,batch_size=50,validation_data=(x_val,y_val))

Epoch 1/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.2440 - loss: 1.4837 - val_accuracy: 0.3667 - val_loss: 1.7153
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 134ms/step - accuracy: 0.3807 - loss: 1.2814 - val_accuracy: 0.3833 - val_loss: 1.1039
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 131ms/step - accuracy: 0.4969 - loss: 1.1406 - val_accuracy: 0.6167 - val_loss: 0.9465
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 131ms/step - accuracy: 0.5462 - loss: 1.0777 - val_accuracy: 0.7333 - val_loss: 0.8475
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 131ms/step - accuracy: 0.6178 - loss: 0.9936 - val_accuracy: 0.6833 - val_loss: 0.9060
Epoch 6/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 129ms/step - accuracy: 0.6618 - loss: 0.8672 - val_accuracy: 0.6833 - val_loss: 0.8723
Epoch 7/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 132ms/step - accuracy: 0.7287 - loss: 0.7083 - val_accuracy: 0.6667 - val_loss: 0.8127
Epoch 8/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 132ms/step - accuracy: 0.8149 - loss: 0.6156 - val_accuracy: 0.7167 - val_loss: 0

In [ ]:
score = model.evaluate(x_val,y_val)

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 423ms/step - accuracy: 0.7826 - loss: 1.6069


In [ ]:
print(f'Test loss: {score[0]}')
print(f'Test accuracy: {score[1] * 100}')

Test loss: 1.7134966850280762
Test accuracy: 78.33333611488342


In [ ]:
import os
if not os.path.exists("C:/Users/Admin/autogradder/scoring.flag"):
    # YOUR CODE HERE
    # raise NotImplementedError()
    model.save("model.h5")
else:
    print("load model")
    model = load_model("model.h5")

In [ ]:
save_model_path = '/content/drive/MyDrive/DeepLearning/Colab/ObjectDetection/model.h5'
model.save(save_model_path)

Reference: https://www.kaggle.com/code/projectang/fruit-image-detection